In [29]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain_community.document_loaders import TextLoader

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough


# Load documents

Load documents to do question answering over. If you want to do this over your documents, this is the section you should replace.



In [5]:
loader = TextLoader('state_of_the_union.txt')
documents = loader.load()

# Split documents

Split documents into small chunks. This is so we can find the most relevant chunks for a query and pass only those into the LLM.

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

TypeError: TextSplitter.split_documents() got an unexpected keyword argument 'persist_directory'

In [9]:
len(texts)

43

# Initialize ChromaDB

Create embeddings for each chunk and insert into the Chroma vector database.


In [14]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectordb = Chroma.from_documents(texts, embeddings,persist_directory="./chroma_db")


In [15]:
docs = vectordb.similarity_search("What did the president say about Ketanji Brown Jackson")

In [17]:
len(docs)

4

# RAG

In [64]:
# LLM prompt template
template = """You are an assistant for specific knowledge query tasks. 
   Use the following pieces of retrieved context to answer the question. 
   If you don't know the answer, just say that you don't know. 
   Question: {question} 
   Context: {context} 
   Answer:
   """

# RAG prompt
template1 = """Answer the question based only on the following context:
{context}
Question: {question}
"""

In [65]:
# supports many more optional parameters. Hover on your `ChatOllama(...)`
# class to view the latest available supported parameters

llm = ChatOllama(model="phi3")



prompt = ChatPromptTemplate.from_template(template)


In [57]:
retriever = vectordb.as_retriever(k=3)

In [70]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [66]:
# RAG

chain = (
    RunnableParallel({"context": retriever | format_docs, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

see link below how to return the sources
https://python.langchain.com/v0.1/docs/use_cases/question_answering/sources/

In [78]:
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

chain = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

# Ask

In [79]:

res = chain.invoke("how much will intel will invest?")

In [94]:
res

{'context': [Document(page_content='Up to eight state-of-the-art factories in one place. 10,000 new good-paying jobs. \n\nSome of the most sophisticated manufacturing in the world to make computer chips the size of a fingertip that power the world and our everyday lives. \n\nSmartphones. The Internet. Technology we have yet to invent. \n\nBut that’s just the beginning. \n\nIntel’s CEO, Pat Gelsinger, who is here tonight, told me they are ready to increase their investment from  \n$20 billion to $100 billion. \n\nThat would be one of the biggest investments in manufacturing in American history. \n\nAnd all they’re waiting for is for you to pass this bill. \n\nSo let’s not wait any longer. Send it to my desk. I’ll sign it.  \n\nAnd we will really take off. \n\nAnd Intel is not alone. \n\nThere’s something happening in America. \n\nJust look around and you’ll see an amazing story. \n\nThe rebirth of the pride that comes from stamping products “Made In America.” The revitalization of Ameri